# Marginal Social Costs for Health-Related Emissions

In [ ]:
import os
import pandas as pd

# import from cmu-tare-model package
from config import PROJECT_ROOT
from cmu_tare_model.utils.inflation_adjustment import cpi_ratio_2023_2006

def process_rcm_data(filename, PROJECT_ROOT):
    """
    Processes the RCM data for use in creating a health-related emissions marginal social cost lookup dictionary.
    """
    # Step 1: Load CSV
    relative_path = os.path.join("cmu_tare_model", "data", "marginal_social_costs", filename)
    file_path = os.path.join(PROJECT_ROOT, relative_path)
    df_rcm_msc_data = pd.read_csv(file_path)

    # Step 2: Filter rows where the season column is 'annual'
    df_rcm_msc_data = df_rcm_msc_data[df_rcm_msc_data['season'] == 'annual']

    # Step 3: Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)
    df_rcm_msc_data['fips'] = df_rcm_msc_data['fips'].astype(str).str.zfill(5)

    # Step 4: Inflate from USD2006 to USD2023
    # CACES Default VSL is 7.4M USD2006
    # We took the New EPA VSL 11.3 

    # Step 4: Return df_rcm_msc_data so that it can be processed into a nested lookup dictionary
    return df_rcm_msc_data

def create_lookup_nested(df_rcm_msc_data):
    """
    Creates a nested lookup dictionary with keys as (fips, state_abbr),
    then model, then pollutant mapping to damage.
    
    Assumptions:
      - 'fips' is a padded string,
      - 'state_abbr' is provided,
      - 'model' identifies the damage model,
      - 'pollutant' is already in uppercase.

    ======================================================================================================================
    LOOKUP DICTIONARY FORMAT
    ======================================================================================================================
    lookup = {
        (fips, state_abbr): {
            model: {
                pollutant: damage,
                ...
            },
            ...
        },
        ...
    }
    ======================================================================================================================
    """
    lookup_health_rcm_msc = {}
    for _, row in df_rcm_msc_data.iterrows():
        county_key = (row['fips'], row['state_abbr'])
        model = row['model']
        pollutant = row['pollutant']
        damage = row['damage']
        
        # Initialize dictionary for the county if needed.
        if county_key not in lookup_health_rcm_msc:
            lookup_health_rcm_msc[county_key] = {}
        # Initialize dictionary for the model if needed.
        if model not in lookup_health_rcm_msc[county_key]:
            lookup_health_rcm_msc[county_key][model] = {}
        # Set the damage value for the pollutant.
        lookup_health_rcm_msc[county_key][model][pollutant] = damage
        
    return lookup_health_rcm_msc

Project root directory: c:\users\14128\research\cmu-tare-model
Project root directory: c:\users\14128\research\cmu-tare-model


## Fossil Fuels are Ground-Level Emissions

In [ ]:
# ======================================================================================================================
# FOSSIL FUEL COMBUSTION:
# Load the data for Fossil Fuel Marginal Social Costs 
# Emissions from Fossil Fuels are considered Ground-Level
# ======================================================================================================================

# Fossil Fuels
# Ground Level, ACS C-R Function
df_ground_acs = process_rcm_data("rcm_msc_county_vsl857_usd2006_ground_acs.csv", PROJECT_ROOT)

# Ground Level, H6C C-R Function
df_ground_h6c = process_rcm_data("rcm_msc_county_vsl857_usd2006_ground_h6c.csv", PROJECT_ROOT)

print(f"""
======================================================================================================================
FOSSIL FUEL COMBUSTION:
Load the data for Fossil Fuel Marginal Social Costs 
Emissions from Fossil Fuels are considered Ground-Level
======================================================================================================================
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: ground level

C-R Function: ACS or H6C
======================================================================================================================
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Ground Level, ACS C-R Function

{df_ground_acs}
      
DATAFRAME: Ground Level, H6C C-R Function

{df_ground_h6c}
      
""")


FOSSIL FUEL COMBUSTION:
Load the data for Fossil Fuel Marginal Social Costs 
Emissions from Fossil Fuels are considered Ground-Level
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: ground level

C-R Function: ACS or H6C
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Ground Level, ACS C-R Function

       fips state_abbr pollutant   model  season      elevated        damage
0     01001         AL      pm25     AP2  annual  ground level  31153.706301
1     01001         AL       so2     AP2  annual  ground level  17013.520087
2     01001         AL       nox     AP2  annual  ground level   2253.330439
3     01001         AL      pm25  EASIUR  annual  ground level  89706.549246
4     01001         AL       so2  EASIUR  annual  ground level  17258.374396
...     ...        ...       ...  

In [3]:
# ======================================================================================================================
# MARGINAL SOCIAL COSTS LOOKUP DICTIONARY: FOSSIL FUEL COMBUSTION
# ======================================================================================================================

# Ground Level, ACS C-R Function
lookup_health_rcm_ground_acs = create_lookup_nested(df_ground_acs)

# Ground Level, H6C C-R Function
lookup_health_rcm_ground_h6c = create_lookup_nested(df_ground_h6c)


print(f"""
======================================================================================================================
LOOKUP DICTIONARIES: 
======================================================================================================================

LOOKUP: Ground Level, ACS C-R Function
      
{lookup_health_rcm_ground_acs}
      
LOOKUP: Ground Level, H6C C-R Function

{lookup_health_rcm_ground_h6c}

""")


LOOKUP DICTIONARIES: 

LOOKUP: Ground Level, ACS C-R Function
      
{('01001', 'AL'): {'AP2': {'pm25': 31153.7063011926, 'so2': 17013.520087235793, 'nox': 2253.3304392998903}, 'EASIUR': {'pm25': 89706.54924600964, 'so2': 17258.37439599273, 'nox': 4187.920973990229}, 'InMAP': {'pm25': 44913.4971871108, 'so2': 16263.359860330058, 'nox': 7192.0377100639}}, ('01003', 'AL'): {'AP2': {'pm25': 72624.10847562253, 'so2': 31784.256532804087, 'nox': 2250.2850454792056}, 'EASIUR': {'pm25': 88666.67635228984, 'so2': 16139.798252975937, 'nox': 3319.9383141067456}, 'InMAP': {'pm25': 63830.69648761314, 'so2': 17135.58922453532, 'nox': 9793.76910058851}}, ('01005', 'AL'): {'AP2': {'pm25': 25571.957357753094, 'so2': 16932.469700693557, 'nox': 2457.190894948541}, 'EASIUR': {'pm25': 83249.94725995786, 'so2': 16190.922095323967, 'nox': 3715.048343166985}, 'InMAP': {'pm25': 26836.65233350246, 'so2': 15278.75388383336, 'nox': 6148.258013134476}}, ('01007', 'AL'): {'AP2': {'pm25': 29086.98549050461, 'so2': 

## Emissions from Electricity Generation are considered "Elevated" or "High-Stack"

In [4]:
# ======================================================================================================================
# ELECTRICITY GENERATION:
# Load the data for Electricity Marginal Social Costs 
# Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
# ======================================================================================================================

# Fossil Fuels
# Ground Level, ACS C-R Function
df_elevated_acs = process_rcm_data("rcm_msc_county_vsl857_usd2006_elevated_acs.csv", PROJECT_ROOT)

# Ground Level, H6C C-R Function
df_elevated_h6c = process_rcm_data("rcm_msc_county_vsl857_usd2006_elevated_h6c.csv", PROJECT_ROOT)

print(f"""
======================================================================================================================
Load the data for Electricity Marginal Social Costs
Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
======================================================================================================================
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: high stack

C-R Function: ACS or H6C
======================================================================================================================
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Elevated (High Stack), ACS C-R Function

{df_elevated_acs}
      
DATAFRAME: Elevated (High Stack), H6C C-R Function

{df_elevated_h6c}
      
""")


Load the data for Electricity Marginal Social Costs
Emissions from Electricity Generation are considered "Elevated" or "High-Stack"
Model: RCM
Output Area: FIPS
Geography: Counties
Pollutant: NOX,SO2,PM25
VSL: 8.57M (Inflated from 11.3M USD2021 to 8.57M USD2006)
Stack Height: high stack

C-R Function: ACS or H6C
- Clean the data so that only annual values are retained. 
- Convert 'fips' to string and pad with leading zeros (to ensure a 5-digit format)

DATAFRAME: Elevated (High Stack), ACS C-R Function

       fips state_abbr pollutant   model  season    elevated        damage
0     01001         AL      pm25     AP2  annual  high stack  17467.703269
1     01001         AL       so2     AP2  annual  high stack  13603.292250
2     01001         AL       nox     AP2  annual  high stack   2130.704503
3     01001         AL      pm25  EASIUR  annual  high stack  47246.623498
4     01001         AL       so2  EASIUR  annual  high stack  16113.086572
...     ...        ...       ...     ...

In [5]:
# ======================================================================================================================
# MARGINAL SOCIAL COSTS LOOKUP DICTIONARY: ELECTRICITY GENERATION
# ======================================================================================================================

# Ground Level, ACS C-R Function
lookup_health_rcm_elevated_acs = create_lookup_nested(df_elevated_acs)

# Ground Level, H6C C-R Function
lookup_health_rcm_elevated_h6c = create_lookup_nested(df_elevated_h6c)


print(f"""
======================================================================================================================
LOOKUP DICTIONARIES: 
======================================================================================================================

LOOKUP: Elevated (High Stack), ACS C-R Function
      
{lookup_health_rcm_elevated_acs}
      
LOOKUP: Elevated (High Stack), H6C C-R Function

{lookup_health_rcm_elevated_h6c}

""")


LOOKUP DICTIONARIES: 

LOOKUP: Elevated (High Stack), ACS C-R Function
      
{('01001', 'AL'): {'AP2': {'pm25': 17467.70326871646, 'so2': 13603.292249828888, 'nox': 2130.704502837654}, 'EASIUR': {'pm25': 47246.62349822275, 'so2': 16113.086571989728, 'nox': 2516.2648986597956}, 'InMAP': {'pm25': 22893.256492958084, 'so2': 15197.449526988015, 'nox': 5111.169577094593}}, ('01003', 'AL'): {'AP2': {'pm25': 22144.176075311378, 'so2': 13639.481519616043, 'nox': 1948.3123123560736}, 'EASIUR': {'pm25': 45650.97285766424, 'so2': 15467.353675802591, 'nox': 2245.018396719472}, 'InMAP': {'pm25': 27696.024513120745, 'so2': 16481.42779323927, 'nox': 5818.405552180256}}, ('01005', 'AL'): {'AP2': {'pm25': 15807.947624907458, 'so2': 14141.382275567845, 'nox': 2046.0675326160308}, 'EASIUR': {'pm25': 44427.34217080605, 'so2': 16397.68868968673, 'nox': 2364.6535450298065}, 'InMAP': {'pm25': 20918.616107042504, 'so2': 14001.125436045872, 'nox': 4885.33128996626}}, ('01007', 'AL'): {'AP2': {'pm25': 15883.2